In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings

# Load the dataset
data = pd.read_csv(r"D:\DS\Neural networks\Alphabets_data.csv")

# Separating features and labels
X = data.drop('letter', axis=1)  # Features are all columns except 'letter'
y = data['letter']  # Label is the 'letter' column

# Encoding the categorical labels into numerical format
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Function to create and compile model
def create_model(optimizer='adam', neurons=64):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(neurons // 2, activation='relu'))
    model.add(Dense(len(np.unique(y_encoded)), activation='softmax'))
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Hyperparameter options
optimizers = ['adam', 'rmsprop']
neurons = [32, 64, 128]
best_accuracy = 0
best_params = {}

# Manual grid search
for optimizer in optimizers:
    for neuron in neurons:
        model = create_model(optimizer=optimizer, neurons=neuron)
        model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=0)
        
        # Evaluate on the test set
        y_pred = np.argmax(model.predict(X_test), axis=1)
        accuracy = accuracy_score(y_test, y_pred)
        
        print(f'Optimizer: {optimizer}, Neurons: {neuron}, Accuracy: {accuracy:.4f}')
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'optimizer': optimizer, 'neurons': neuron}

print(f'Best Accuracy: {best_accuracy:.4f} with parameters: {best_params}')


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Optimizer: adam, Neurons: 32, Accuracy: 0.8792
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Optimizer: adam, Neurons: 64, Accuracy: 0.9367
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Optimizer: adam, Neurons: 128, Accuracy: 0.9453
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Optimizer: rmsprop, Neurons: 32, Accuracy: 0.8822
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Optimizer: rmsprop, Neurons: 64, Accuracy: 0.9210
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Optimizer: rmsprop, Neurons: 128, Accuracy: 0.9480
Best Accuracy: 0.9480 with parameters: {'optimizer': 'rmsprop', 'neurons': 128}


In [11]:

import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings# Train the final model using the best hyperparameters
best_optimizer = best_params['optimizer']
best_neurons = best_params['neurons']

final_model = create_model(optimizer=best_optimizer, neurons=best_neurons)

# Train the final model on the entire training set
final_model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)


Epoch 1/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5445 - loss: 1.6845
Epoch 2/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8176 - loss: 0.6015
Epoch 3/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8755 - loss: 0.4128
Epoch 4/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9012 - loss: 0.3245
Epoch 5/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9157 - loss: 0.2708
Epoch 6/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9285 - loss: 0.2296
Epoch 7/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9363 - loss: 0.2013
Epoch 8/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9423 - loss: 0.1821
Epoch 9/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9482 - loss: 0.1595
Epoch 10/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9495 - loss: 0.1526
Epoch 11/50
1600/1600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9577 - loss: 0.1374
Epoch 12/50
1600/1600 ━━━━━━━━

In [7]:
# Evaluate the final model on the test set
y_pred_final = np.argmax(final_model.predict(X_test), axis=1)

accuracy_final = accuracy_score(y_test, y_pred_final)
precision_final = precision_score(y_test, y_pred_final, average='weighted')
recall_final = recall_score(y_test, y_pred_final, average='weighted')
f1_final = f1_score(y_test, y_pred_final, average='weighted')

print(f'Final Model Accuracy: {accuracy_final:.2f}')
print(f'Final Model Precision: {precision_final:.2f}')
print(f'Final Model Recall: {recall_final:.2f}')
print(f'Final Model F1 Score: {f1_final:.2f}')


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Final Model Accuracy: 0.95
Final Model Precision: 0.95
Final Model Recall: 0.95
Final Model F1 Score: 0.95


In [15]:
# Save the final model

final_model.save('final_alphabet_classifier_model.h5')
print("Model saved to disk.")


Model saved to disk.


In [16]:
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings
# Load the saved model
loaded_model = load_model('final_alphabet_classifier_model.h5')

# Use the loaded model for predictions
y_pred_loaded = np.argmax(loaded_model.predict(X_test), axis=1)

# Verify that the loaded model gives the same performance
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
print(f'Loaded Model Accuracy: {accuracy_loaded:.2f}')


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Loaded Model Accuracy: 0.96
